In [1]:
import hmac
import hashlib
import base64

import bitarray
import pybloom_live

import xml.etree.ElementTree as ET
from comarmor.xml.utils import beautify_xml

In [2]:
def obscure_permissions(path, capacity=100, error_rate=0.001):
    tree = ET.parse(path)
    root = tree.getroot()
#     root = ET.fromstring(path)
    permissions = root.find('permissions')
    for grant in permissions.findall('grant'):
        obscure_grant(
            grant,
            capacity=capacity,
            error_rate=error_rate)
    return tree
#     return ET.ElementTree(root)


def obscure_grant(grant, capacity, error_rate):
    subject_name = grant.find('subject_name')
    for rule in grant.findall('*/domains/..'):
        obscure_rule(
            rule=rule,
            capacity=capacity,
            error_rate=error_rate)


def obscure_rule(rule, capacity, error_rate):
    obscure_topics(rule, capacity, error_rate)
    obscure_partitions(rule, capacity, error_rate)
    obscure_data_tags(rule, capacity, error_rate)


def obscure_topics(rule, capacity, error_rate):
    for criteria in rule.findall('*/topics/..'):
        bf = pybloom_live.BloomFilter(
            capacity=capacity,
            error_rate=error_rate)
        for element in criteria.findall('topics/topic'):
            bf.add(element.text.encode('utf-8'))
        element = criteria.find('topics')
        element.clear()
        digest_base64 = base64.b64encode(bf.bitarray.tobytes())
        element.text = digest_base64.decode('utf-8')


def obscure_partitions(rule, capacity, error_rate):
    for criteria in rule.findall('*/partitions/..'):
        bf = pybloom_live.BloomFilter(
            capacity=capacity,
            error_rate=error_rate)
        for element in criteria.findall('partitions/partition'):
            bf.add(element.text.encode('utf-8'))
        element = criteria.find('partitions')
        element.clear()
        digest_base64 = base64.b64encode(bf.bitarray.tobytes())
        element.text = digest_base64.decode('utf-8')


def obscure_data_tags(rule, capacity, error_rate):
    for criteria in rule.findall('*/data_tags/..'):
        bf = pybloom_live.BloomFilter(
            capacity=capacity,
            error_rate=error_rate)
        for element in criteria.findall('data_tags/tag'):
            name = element.find('name')
            value = element.find('value')
            bf.add((name.text + ':' + value.text).encode('utf-8'))
        element = criteria.find('data_tags')
        element.clear()
        digest_base64 = base64.b64encode(bf.bitarray.tobytes())
        element.text = digest_base64.decode('utf-8')

In [3]:
permissions_file_path = '/home/ruffsl/Desktop/keymint_ws/build/talker/permissions.xml'

# import urllib.request
# some_url = 'https://www.omg.org/spec/DDS-SECURITY/20170901/omg_shared_ca_permissions_example.xml'
# with urllib.request.urlopen(some_url) as url:
#     permissions_file_path = url.read()

In [4]:
tree = obscure_permissions(permissions_file_path)

In [5]:
print(beautify_xml(tree.getroot()))

<?xml version="1.0" encoding="utf-8"?>
<dds>
  <permissions>
    <grant name="talker">
      <subject_name>CN=talker</subject_name>
      <validity>
        <not_before>2013-10-26T00:00:00</not_before>
        <not_after>2023-10-26T22:45:30</not_after>
      </validity>
      <allow_rule>
        <domains>
          <id_range>
            <min>0</min>
            <max>42</max>
          </id_range>
        </domains>
        <subscribe>
          <topics>AQBAAEIMAAgAAAAIIAAmACAEQAAAECAAgAAQAAAsAACGQCAAAAAQQQABQCAAQAEEAAAIAQQAACBAAAAEgkAAJAIACRABAAEAAAAwAEQsEEAAAFIEAAAAgAAABQABEAAoBABIAAAA4BAAAIAAAACAIACACAAgCAgSMAAAAQAEAACAAgAACAAwQEAAAQJAAIAYIAEAAAAAIAEEAgIAGAIIgAABAAAMAAAiAIBAQAAoAAAgJABA</topics>
        </subscribe>
        <publish>
          <topics>AABgAEIMAAgAAAAIIAAmACAEQAAAECAAgAAQAAAsAAKCQCAAAgAQQQAAQCAAQAEEAAAIAQQAACBAAAAEgkAAJAAACRABEAEAAAAwAEQkEEAAAFIEAAAAgAAABQABEAAoBABIIAAA4BAAAAAAAACAIACAABAgCAgSMAAAAQAEAACAAgBACAAgQEAAAQJAAIAYIAEAAAAAIAEEAgAAOAIIgAABAAAEAAAiAIBBQAAoAA

In [6]:
bf = pybloom_live.BloomFilter(capacity=100, error_rate=0.001)
bf.bitarray = bitarray.bitarray(endian='little')

root = tree.getroot()
bf_str = root.find('permissions/grant/allow_rule/subscribe/topics').text
bf.bitarray.frombytes(base64.decodebytes(bf_str.encode('utf-8')))

In [7]:
topic = 'rt/clock'.encode('utf-8')
topic in bf

True